# Project Overview

These 6 exercises are done in SQLite, to demonstrate practice writing simple SQL
reports from the perspective of a business analyst at a fictional school,
Jefferson Middle School.  This coursework is from [Treehouse](http://teamtreehouse.com) **SQL Reporting By Example**.

To view the 6 queries in a stand-alone .sql file, open the file `6-queries.sql` in this repository.

### Initializing the SQLite database

Please see the SQL commands to create this database in the file `school-system-db-setup.sql` in this repository.
Alternatively, you can find it at [this AWS S3 URL](https://s3.amazonaws.com/treehouse-code-samples/sql_reporting_by_example.txt)

## Database Overview

The database at Jefferson Middle School contains the following seven related tables.

```
STUDENTS
TEACHERS
SUBJECTS
ROOMS
PERIODS
CLASSES
SCHEDULE
```

A detailed description can be viewed in the accompanied file `database-details.md`

## 1) Busiest Teachers

There are 7 periods at Jefferson Middle School.
Which teachers teach a class during all 7 periods?

### query result

```SQL
WITH CLASSES_TEACHERS AS
  (SELECT *
   FROM TEACHERS
   JOIN CLASSES ON TEACHERS.ID = CLASSES.TEACHER_ID)
SELECT TEACHER_ID,
       COUNT(PERIOD_ID) AS NUM_PERIODS
FROM CLASSES_TEACHERS
GROUP BY TEACHER_ID
HAVING NUM_PERIODS = 7;
```

#### report preview

```
TEACHER_ID	FIRST_NAME	LAST_NAME	NUM_PERIODS
373	Jeffrey	Bastion	7
374	Stanley	Petrovic	7
375	Idra	Patel	7
376	Jessica	Dillon	7
377	Patricia	Parker	7
```

Using a Common Table Expression (CTE), the query returns 19 teachers, of which the first 5 are shown here.

## 2) Multi-Subject Teachers

Do any teachers teach multiple subjects?  If so, which teachers?

### query result

```SQL
SELECT DISTINCT FIRST_NAME,
                LAST_NAME,
                NAME AS "CLASS_NAME"
FROM TEACHERS
JOIN CLASSES ON TEACHERS.ID = CLASSES.TEACHER_ID
JOIN SUBJECTS ON SUBJECTS.ID = CLASSES.SUBJECT_ID
WHERE TEACHERS.ID IN
    (SELECT TEACHER_ID
     FROM CLASSES
     GROUP BY TEACHER_ID
     HAVING COUNT(DISTINCT SUBJECT_ID) > 1)
```

#### report preview

```
FIRST_NAME	LAST_NAME	CLASS_NAME
Janis	Ambrose	Band
Janis	Ambrose	Choir
```

The query shows one teacher, Janis Ambrose, who teaches more than one subject.

## 3) Janis' Schedule

What class does Janis Ambrose teach during each period? Be sure to include all 7 periods in your report.

### query result

```SQL
WITH PERIOD_CLASS_SUBJECT_TEACHER AS
  (SELECT *
   FROM PERIODS
   LEFT OUTER JOIN CLASSES ON PERIOD_ID = PERIODS.ID
   LEFT OUTER JOIN SUBJECTS ON SUBJECT_ID = SUBJECTS.ID
   LEFT OUTER JOIN TEACHERS ON TEACHER_ID = TEACHERS.ID
   WHERE FIRST_NAME = "Janis"
     AND LAST_NAME = "Ambrose" )
SELECT PERIODS.ID AS "Period No.",
       ROOM_ID,
       NAME AS "Subjects_Janis_Teaches"
FROM PERIODS
LEFT OUTER JOIN PERIOD_CLASS_SUBJECT_TEACHER ON PERIODS.ID = PERIOD_ID;
```

#### report preview

```
Period No.	ROOM_ID	Subjects_Janis_Teaches
1	19	Band
2	19	Band
3	19	Band
4		
5	19	Choir
6	19	Choir
7	19	Choir
```

Using a series of outer joins, in additional to another CTE, we see that Janis' 4th period is open, when she teaches no classes.


## 4) Least Popular Subject

Which subject is the least popular, and how many students are taking it?

### query result

```SQL
WITH CTE AS
  (SELECT NAME,
          COUNT(1) AS "STUDENT_COUNT"
   FROM SUBJECTS
   JOIN CLASSES ON SUBJECT_ID = SUBJECTS.ID
   JOIN SCHEDULE ON CLASS_ID = CLASSES.ID
   GROUP BY SUBJECT_ID)
SELECT NAME,
       MIN("STUDENT_COUNT")
FROM CTE;
```

#### report preview

```
NAME	MIN("STUDENT_COUNT")
Puppetry	58
```

Puppetry is the least popular subject; only 58 students total in attendance.

## 5) Most Popular Teacher

Which elective teacher is the most popular (i.e., which teacher teaches the most students)?

### query result

```SQL
SELECT TEACHERS.FIRST_NAME || " " || TEACHERS.LAST_NAME AS "Teacher",
       COUNT(SCHEDULE.STUDENT_ID) AS "Total_Student_Count"
FROM SUBJECTS
JOIN CLASSES ON CLASSES.SUBJECT_ID = SUBJECTS.ID
JOIN TEACHERS ON TEACHERS.ID = CLASSES.TEACHER_ID
JOIN SCHEDULE ON CLASSES.ID = SCHEDULE.CLASS_ID
WHERE SUBJECTS.GRADE IS NULL
GROUP BY CLASSES.TEACHER_ID
ORDER BY "Total_Student_Count" DESC
LIMIT 1;
```

#### report preview

```
Teacher	Total_Student_Count
Janis Ambrose	140
```

It would appear that Janis Ambrose appears again as the most popular teacher, teaching 140 students.

## 6) No 1st Period Class

Which teachers don't have a class during 1st period?

### query result

```SQL
WITH free_teachers AS
  (SELECT TEACHERS.ID
   FROM TEACHERS
   WHERE TEACHERS.ID NOT IN
       (SELECT TEACHER_ID
        FROM CLASSES
        WHERE PERIOD_ID=1 ) )
SELECT free_teachers.ID,
       CLASSES.PERIOD_ID,
       TEACHERS.*
FROM free_teachers
JOIN CLASSES ON CLASSES.TEACHER_ID = free_teachers.ID
JOIN TEACHERS ON TEACHERS.ID = free_teachers.ID
```

#### report preview

```
free_teachers.ID	CLASSES.PERIOD_ID	ID	FIRST_NAME	LAST_NAME
378	2	378	Cassandra	McGuire
378	3	378	Cassandra	McGuire
378	4	378	Cassandra	McGuire
378	5	378	Cassandra	McGuire
378	6	378	Cassandra	McGuire
378	7	378	Cassandra	McGuire
```

Cassandra McGuire is the only teacher without a 1st period class scheduled.